## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

In [ ]:
#!pip install transformers #in case of a Greek models?

# Data


In [ ]:
import pandas as pd

In [ ]:
#connect to G-Drive
from google.colab import drive
mount='/content/gdrive/'
drive.mount(mount, force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%cd "/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling"

/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling


In [ ]:
df = pd.read_csv("process-steps-full.csv", sep=',',
    usecols=[0,1,2,3,4,5,6],
    encoding="utf8")

df = df.dropna(subset=['step_title']) #drop also empty values (had 2 -> quality issue)

docs = df["step_title"].values.tolist()

In [ ]:
len(docs)

19931

# **Topic Modeling**



## Training

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>20_000 documents). It is advised to turn this off if you want to speed up the model.


### Stop Words for Count Vectorizer

In [ ]:
# get Greek stop_words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
greek_stopwords = stopwords.words('greek')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
new_words = ['της', 'τη', 'του', 'από']

for word in new_words:
  greek_stopwords.append(word)

### Initialize models & Training

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # Embeddings
from umap import UMAP #Dimensionality reduction
from hdbscan import HDBSCAN #clustering
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from transformers.pipelines import pipeline

As the number of sample increases the set-up of parameters might have to change in order to have good results. So there is a trail & error game


In [ ]:
#embedding_model = pipeline("feature-extraction", model="nlpaueb/bert-base-greek-uncased-v1") #There is also the option of greek model, but after some test the other model worked better
embedding_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
umap_model = UMAP(n_neighbors=100, n_components=10, min_dist=0.5, metric='cosine') #increasing n_neighbors in general larger clusters | n_components - in which dimension. Too small loss of info, too high performance | min_dist how far the points should be in low dimensional
hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom') #min_cluster_size, increasing -> fewer clusters | cluster_selection_method also the "leaf" method available (smaller clusters)
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=greek_stopwords) #ngram_range combination of words
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)

In [ ]:
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(docs)

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8596,-1_ελέγχου_ενημέρωση_στοιχείων_εγγράφου,"[ελέγχου, ενημέρωση, στοιχείων, εγγράφου, κοιν...","[Δημοσίευση στο Μητρώο Αιτήσεων Αδειών, Δημοσί..."
1,0,603,0_ποσού_πληρωμής_πληρωμή_χρηματικού,"[ποσού, πληρωμής, πληρωμή, χρηματικού, οικονομ...","[Αποστολή Αποδεικτικών Πληρωμής, Αποστολή Αποδ..."
2,1,593,1_εκπαίδευσης_δευτεροβάθμιας_δευτεροβάθμιας εκ...,"[εκπαίδευσης, δευτεροβάθμιας, δευτεροβάθμιας ε...",[Διαβίβαση των αιτήσεων μετάθεσης εκπαιδευτικώ...
3,2,423,2_άδειας_κατηγορίας_εμπορίας_άδειας εμπορίας,"[άδειας, κατηγορίας, εμπορίας, άδειας εμπορίας...",[Τροποποίηση ως προς την επωνυμία κατόχου Άδει...
4,3,419,3_διαδικασίας εκκίνηση_διαδικασίας εκκίνηση δι...,"[διαδικασίας εκκίνηση, διαδικασίας εκκίνηση δι...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
5,4,210,4_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος δικαιολογητικών, Έλεγχος δικαιολογητι..."
6,5,203,5_υπάλληλο_υπηρεσίας_ανάθεση_ανάθεση αρμόδιο,"[υπάλληλο, υπηρεσίας, ανάθεση, ανάθεση αρμόδιο...","[Πρωτοκόλληση και ανάθεση σε αρμόδιο υπάλληλο,..."
7,6,194,6_δικαιολογητικών παραλαβή δικαιολογητικών_παρ...,"[δικαιολογητικών παραλαβή δικαιολογητικών, παρ...","[Παραλαβή δικαιολογητικών, Παραλαβή δικαιολογη..."
8,7,193,7_απορριπτικής απόφασης_απορριπτικής_έκδοση απ...,"[απορριπτικής απόφασης, απορριπτικής, έκδοση α...","[Έκδοση απορριπτικής απόφασης, Έκδοση απορριπτ..."
9,8,183,8_ελληνικής_ελληνικής αστυνομίας_αστυνομίας_αρ...,"[ελληνικής, ελληνικής αστυνομίας, αστυνομίας, ...","[Απόφαση του Αρχηγού της Ελληνικής Αστυνομίας,..."


In [ ]:
freq["Topic"].nunique()

141

-1 refers to all outliers. But this amount is quite high, mote that 25% of out data. So we would like to reduce the noise. Closely related items will be assigned in the same topic. And in that topic they would be recognised as unique in the similarity process

In [ ]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings", threshold=0.70)

In [ ]:
topic_model.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2547,-1_εγγραφή_επιτροπή_προς_μέσω,"[εγγραφή, επιτροπή, προς, μέσω, υπηρεσίες, μελ...","[Δημοσίευση στο Μητρώο Αιτήσεων Αδειών, Δημοσί..."
1,0,607,0_ποσού_πληρωμής_πληρωμή_χρηματικού,"[ποσού, πληρωμής, πληρωμή, χρηματικού, οικονομ...","[Αποστολή Αποδεικτικών Πληρωμής, Αποστολή Αποδ..."
2,1,601,1_εκπαίδευσης_δευτεροβάθμιας_δευτεροβάθμιας εκ...,"[εκπαίδευσης, δευτεροβάθμιας, δευτεροβάθμιας ε...",[Διαβίβαση των αιτήσεων μετάθεσης εκπαιδευτικώ...
3,2,476,2_άδειας_εμπορίας_κατηγορίας_άδειας εμπορίας,"[άδειας, εμπορίας, κατηγορίας, άδειας εμπορίας...",[Τροποποίηση ως προς την επωνυμία κατόχου Άδει...
4,3,423,3_διαδικασίας εκκίνηση_διαδικασίας εκκίνηση δι...,"[διαδικασίας εκκίνηση, διαδικασίας εκκίνηση δι...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
5,4,236,4_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος δικαιολογητικών, Έλεγχος δικαιολογητι..."
6,5,300,5_υπάλληλο_υπηρεσίας_ανάθεση_ανάθεση αρμόδιο,"[υπάλληλο, υπηρεσίας, ανάθεση, ανάθεση αρμόδιο...","[Πρωτοκόλληση και ανάθεση σε αρμόδιο υπάλληλο,..."
7,6,220,6_παραλαβή δικαιολογητικών_δικαιολογητικών παρ...,"[παραλαβή δικαιολογητικών, δικαιολογητικών παρ...","[Παραλαβή δικαιολογητικών, Παραλαβή δικαιολογη..."
8,7,212,7_αρνητική_απορριπτικής απόφασης_απορριπτικής_...,"[αρνητική, απορριπτικής απόφασης, απορριπτικής...","[Έκδοση απορριπτικής απόφασης, Έκδοση απορριπτ..."
9,8,195,8_ελληνικής_ελληνικής αστυνομίας_αστυνομίας_αρ...,"[ελληνικής, ελληνικής αστυνομίας, αστυνομίας, ...","[Απόφαση του Αρχηγού της Ελληνικής Αστυνομίας,..."


In [ ]:
freq["Topic"].nunique()

141

We reduced a lot the outliers to 10%, which would be good for our semantic analysis.  Finally have a look if we should merge some topics

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topics_to_merge = [[4,38]] #seem to close
merged_topics = topic_model.merge_topics(docs, topics_to_merge)

In [ ]:
topic_model.update_topics(docs, topics=merged_topics, vectorizer_model=vectorizer_model,  ctfidf_model=ctfidf_model)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2547,-1_εγγραφή_επιτροπή_προς_μέσω,"[εγγραφή, επιτροπή, προς, μέσω, υπηρεσίες, μελ...",[Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτί...
1,0,607,0_ποσού_πληρωμής_πληρωμή_χρηματικού,"[ποσού, πληρωμής, πληρωμή, χρηματικού, οικονομ...","[Αποστολή Αποδεικτικών Πληρωμής, Αποστολή Αποδ..."
2,1,601,1_εκπαίδευσης_δευτεροβάθμιας_δευτεροβάθμιας εκ...,"[εκπαίδευσης, δευτεροβάθμιας, δευτεροβάθμιας ε...",[Διαβίβαση των αιτήσεων μετάθεσης εκπαιδευτικώ...
3,2,476,2_άδειας_εμπορίας_κατηγορίας_άδειας εμπορίας,"[άδειας, εμπορίας, κατηγορίας, άδειας εμπορίας...","[Χορήγηση Άδειας Εμπορίας Κατηγορίας Γ, Χορήγη..."
4,3,423,3_διαδικασίας εκκίνηση_διαδικασίας εκκίνηση δι...,"[διαδικασίας εκκίνηση, διαδικασίας εκκίνηση δι...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε..."
5,4,402,4_αίτησης υποβολή αίτησης_αίτησης υποβολή_υποβ...,"[αίτησης υποβολή αίτησης, αίτησης υποβολή, υπο...","[Υποβολή αίτησης, Υποβολή αίτησης, Υποβολή αίτ..."
6,5,375,5_έλεγχος δικαιολογητικών έλεγχος_δικαιολογητι...,"[έλεγχος δικαιολογητικών έλεγχος, δικαιολογητι...","[Έλεγχος Δικαιολογητικών, Έλεγχος Δικαιολογητι..."
7,6,304,6_αρμόδια_αίτησης αρμόδια_αιτήματος αρμόδια_αρ...,"[αρμόδια, αίτησης αρμόδια, αιτήματος αρμόδια, ...","[Παραλαβή της αίτησης από αρμόδια αρχή, Παραλα..."
8,7,300,7_υπάλληλο_υπηρεσίας_ανάθεση_ανάθεση αρμόδιο,"[υπάλληλο, υπηρεσίας, ανάθεση, ανάθεση αρμόδιο...","[Πρωτοκόλληση - Ανάθεση σε αρμόδιο υπάλληλο, Π..."
9,8,269,8_επιτροπής_συνεδρίαση_τριμελούς_επιτροπή,"[επιτροπής, συνεδρίαση, τριμελούς, επιτροπή, σ...","[Συνεδρίαση της Οικονομικής Επιτροπής, Συνεδρί..."


In [ ]:
freq["Topic"].nunique()

140

And some insights

In [ ]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Φυσική ταυτοποίηση πολίτη στο ΚΕΠ,43,43_ταυτοποίηση πολίτη_φυσική ταυτοποίηση_φυσικ...,"[ταυτοποίηση πολίτη, φυσική ταυτοποίηση, φυσικ...","[Φυσική ταυτοποίηση πολίτη, Φυσική ταυτοποίηση...",ταυτοποίηση πολίτη - φυσική ταυτοποίηση - φυσι...,0.162237,False
1,Υποβολή αίτησης στο Πληροφοριακό σύστημα,26,26_ηλεκτρονική_αριθμού_αριθμού πρωτοκόλλου_ψηφ...,"[ηλεκτρονική, αριθμού, αριθμού πρωτοκόλλου, ψη...",[Παραλαβή αιτήματος και χορήγηση αριθμού πρωτο...,ηλεκτρονική - αριθμού - αριθμού πρωτοκόλλου - ...,0.992964,False
2,Εκτύπωση αποδεικτικού φορολογικής ενημερότητας,0,0_ποσού_πληρωμής_πληρωμή_χρηματικού,"[ποσού, πληρωμής, πληρωμή, χρηματικού, οικονομ...","[Αποστολή Αποδεικτικών Πληρωμής, Αποστολή Αποδ...",ποσού - πληρωμής - πληρωμή - χρηματικού - οικο...,1.000000,False
3,Εκκίνηση διαδικασίας,3,3_διαδικασίας εκκίνηση_διαδικασίας εκκίνηση δι...,"[διαδικασίας εκκίνηση, διαδικασίας εκκίνηση δι...","[Εκκίνηση διαδικασίας, Εκκίνηση διαδικασίας, Ε...",διαδικασίας εκκίνηση - διαδικασίας εκκίνηση δι...,0.789207,True
4,Παραλαβή αιτήματος για διακίνηση χοιροειδών,2,2_άδειας_εμπορίας_κατηγορίας_άδειας εμπορίας,"[άδειας, εμπορίας, κατηγορίας, άδειας εμπορίας...","[Χορήγηση Άδειας Εμπορίας Κατηγορίας Γ, Χορήγη...",άδειας - εμπορίας - κατηγορίας - άδειας εμπορί...,1.000000,False
...,...,...,...,...,...,...,...,...
19926,Πρωτοκόλληση αίτησης,37,37_πρωτοκόλληση αίτησης πρωτοκόλληση_αίτησης π...,"[πρωτοκόλληση αίτησης πρωτοκόλληση, αίτησης πρ...","[Πρωτοκόλληση αίτησης, Πρωτοκόλληση αίτησης, Π...",πρωτοκόλληση αίτησης πρωτοκόλληση - αίτησης πρ...,1.000000,True
19927,Δρομολόγηση αίτησης,105,105_δρομολόγηση αίτησης_δρομολόγηση_αίτησης δρ...,"[δρομολόγηση αίτησης, δρομολόγηση, αίτησης δρο...","[Δρομολόγηση αίτησης, Δρομολόγηση αίτησης, Δρο...",δρομολόγηση αίτησης - δρομολόγηση - αίτησης δρ...,1.000000,True
19928,Έλεγχος έννομου συμφέροντος,102,102_συμφέροντος_έννομου_έννομου συμφέροντος_έλ...,"[συμφέροντος, έννομου, έννομου συμφέροντος, έλ...","[Έλεγχος έννομου συμφέροντος, Έλεγχος έννομου ...",συμφέροντος - έννομου - έννομου συμφέροντος - ...,1.000000,True
19929,Έκδοση πιστοποιητικού,34,34_πιστοποιητικού_έκδοση πιστοποιητικού_πιστοπ...,"[πιστοποιητικού, έκδοση πιστοποιητικού, πιστοπ...","[Έκδοση του πιστοποιητικού, Έκδοση του πιστοπο...",πιστοποιητικού - έκδοση πιστοποιητικού - πιστο...,1.000000,True


In [ ]:
topic_model.get_topic(5)  # Select the most frequent topic

[('έλεγχος δικαιολογητικών έλεγχος', 0.17837772315732628),
 ('δικαιολογητικών έλεγχος δικαιολογητικών', 0.1761199013853915),
 ('έλεγχος δικαιολογητικών', 0.17130767373899813),
 ('δικαιολογητικών έλεγχος', 0.1220712497740064),
 ('αναζήτηση δικαιολογητικών έλεγχος', 0.03498730715102758),
 ('δικαιολογητικών έλεγχος αναζήτηση', 0.03498730715102758),
 ('έλεγχος αναζήτηση δικαιολογητικών', 0.03498730715102758),
 ('έλεγχος αναζήτηση', 0.03498730715102758),
 ('αναζήτηση δικαιολογητικών', 0.03136832959533875),
 ('αναζήτηση', 0.019477201999607167)]

### Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |


## Save Topics

In [ ]:
result = topic_model.get_document_info(docs)

In [ ]:
len(result)

19931

In [ ]:
#add process_id for comparison reasons
result = topic_model.get_document_info(docs)

df = df.reset_index(drop=True) #need also to reindex the df

result['process_id'] = df.loc[result.index, 'process_id']

In [ ]:
result.to_csv('topic_full_strict.csv')

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("έλεγχος δικαιολογητικών", top_n=5); similar_topics

[5, 84, 42, 27, 14]

In [ ]:
topic_model.get_topic(5)

[('έλεγχος δικαιολογητικών έλεγχος', 0.17837772315732628),
 ('δικαιολογητικών έλεγχος δικαιολογητικών', 0.1761199013853915),
 ('έλεγχος δικαιολογητικών', 0.17130767373899813),
 ('δικαιολογητικών έλεγχος', 0.1220712497740064),
 ('αναζήτηση δικαιολογητικών έλεγχος', 0.03498730715102758),
 ('δικαιολογητικών έλεγχος αναζήτηση', 0.03498730715102758),
 ('έλεγχος αναζήτηση δικαιολογητικών', 0.03498730715102758),
 ('έλεγχος αναζήτηση', 0.03498730715102758),
 ('αναζήτηση δικαιολογητικών', 0.03136832959533875),
 ('αναζήτηση', 0.019477201999607167)]

# **Save the model**

In [ ]:
topic_model.save("topic_model_mitos", serialization="pickle")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)